<a href="https://colab.research.google.com/github/NourhanSowar/Spotify-Skip-Action-Prediction-Data-Science-Internship/blob/main/Web_App_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install flask-ngrok
!pip install flask-bootstrap

     |████████████████████████████████| 456 kB 7.4 MB/s 
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460123 sha256=e41d377199531ade7f64b63ea87e9b55a9df1f8c911eee690c316a238726b28c
  Stored in directory: /root/.cache/pip/wheels/67/a2/d6/50d039c9b59b4caca6d7b53839c8100354a52ab7553d2456eb
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3946 sha256=ddc38cad326ce11ef95e7bf9b723a5e224e730fbae7b37927c1124c07a172550
  Stored in directory: /root/.cache/pip/wheels/64/34/11/053f47218984c9a31a00f911ed98dda036b867481dcc527a12
Successfully built flask-bootstrap visitor


In [13]:
!pip install pyngrok
!ngrok authtoken 23EGucliEsZDDf1qn0GOI4DCWYH_4Qm4TrCzhppD1yy6NVUuX

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


* test

In [14]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
	return "<h1>Hello Norhan </h1>"

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2022-01-17 12:27:48.504 INFO    werkzeug:  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://41d4-34-125-221-85.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
from flask_ngrok import run_with_ngrok

import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

app = Flask(__name__)
run_with_ngrok(app)
model = pickle.load(open('model_final.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index2.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    
    tf_mini = pd.read_csv('https://drive.google.com/uc?export=download&id=1Zcd6o-sM0iP5nNZF1FPbOiVpsN6o0KVJ') 
    web_app_scaling_df = pd.read_csv('web_app_scaling_df.csv')      

    form_data = {'session_position' : request.form['session_position'],  # request.form.get('session_position')
                 'session_length' : request.form['session_length'],
                 'track_id' : request.form['track_id'],
                 'context_switch' : request.form['context_switch'],
                 'no_pause_before_play' : request.form['no_pause_before_play'],
                 'hist_user_behavior_n_seekfwd' : request.form['hist_user_behavior_n_seekfwd'],
                 'hist_user_behavior_n_seekback' : request.form['hist_user_behavior_n_seekback'],
                 'hist_user_behavior_is_shuffle' : request.form['hist_user_behavior_is_shuffle'],
                 'hour_of_day' : request.form['hour_of_day'],
                 'date' : request.form['date'],
                 'premium' : request.form['premium'],
                 'context_type' : request.form['context_type'],
                 'hist_user_behavior_reason_start' : request.form['hist_user_behavior_reason_start'],
                 'hist_user_behavior_reason_end' : request.form['hist_user_behavior_reason_end']         }
    
    form_df = pd.DataFrame([list(form_data.values())], columns=list(form_data.keys()))
    
    # Merging form_df & track_df into single dataframe.
    session_track_data = pd.merge(form_df, tf_mini, on='track_id', how='left')
    
    # Replacing boolean (True, False) by int32 (1, 0)
    session_track_data.replace(['Yes', 'No'], [1, 0], inplace=True)
    # encoding the mode
    session_track_data['mode'].replace({'major': 1, 'minor': 0 }, inplace=True)
    
    # chaning the date to weekday and droping the date column
    session_track_data["date"] = pd.to_datetime(session_track_data["date"])
    session_track_data['week_day'] = session_track_data["date"].dt.dayofweek
    session_track_data.drop("date", inplace=True, axis=1)
    
    session_track_data.replace(['playbtn', 'remote', 'trackerror', 'endplay', 'clickrow'], 'merged', inplace=True)
    
    # setting one hot encoding for categorical columns (Nominal Columns)
    One_Hot_Encoder = OneHotEncoder()

    context_type = pd.DataFrame(One_Hot_Encoder.fit_transform(session_track_data[['context_type']]).toarray())
    context_type.columns = One_Hot_Encoder.get_feature_names(['context_type'])

    hist_user_behavior_reason_start = pd.DataFrame(One_Hot_Encoder.fit_transform(session_track_data[['hist_user_behavior_reason_start']]).toarray())
    hist_user_behavior_reason_start.columns = One_Hot_Encoder.get_feature_names(['hub_reason_start']) # hub = hist_user_behavior

    hist_user_behavior_reason_end = pd.DataFrame(One_Hot_Encoder.fit_transform(session_track_data[['hist_user_behavior_reason_end']]).toarray())
    hist_user_behavior_reason_end.columns = One_Hot_Encoder.get_feature_names(['hub_reason_end'])  # hub = hist_user_behavior

    # Concatenate dataframe --> session_track_data + context_type + hist_user_behavior_reason_start + hist_user_behavior_reason_end
    session_track_data = pd.concat([session_track_data, context_type, hist_user_behavior_reason_start, hist_user_behavior_reason_end], axis = 1)

    session_track_data.drop(["context_type", "hist_user_behavior_reason_start", "hist_user_behavior_reason_end", "track_id"], axis = 1, inplace = True)
    
    # drop all highly correlated variables.
    session_track_data.drop(['beat_strength', 'danceability', 'dyn_range_mean'], axis=1, inplace=True)

    web_app_scaling_df.drop(['Unnamed: 0'], axis=1, inplace=True)
    web_app_scaling_df = web_app_scaling_df.append(session_track_data)
    web_app_scaling_df.replace([np.nan], [0], inplace=True)
    web_app_scaling_df.reset_index(drop = True, inplace = True)
    
    # Scaling
    scaler = StandardScaler()
    for col in web_app_scaling_df.columns:
        if (len(web_app_scaling_df[col].unique()) != 2) :
            web_app_scaling_df[col] = scaler.fit_transform(np.array(web_app_scaling_df[col]).reshape(-1, 1))
            
    prediction = model.predict(web_app_scaling_df.tail(1))[0]
    output = ['Not Skip', 'Skip']

    return render_template('index.html', prediction_text= 'Prediction: Probably user will "{}" this Music track.'.format(output[prediction]))

if __name__ == "__main__":
    app.run()
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
2022-01-17 13:08:36.122 INFO    werkzeug:  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ae49-34-125-221-85.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


2022-01-17 13:08:42.513 INFO    werkzeug: 127.0.0.1 - - [17/Jan/2022 13:08:42] "GET / HTTP/1.1" 200 -
2022-01-17 13:08:43.451 INFO    werkzeug: 127.0.0.1 - - [17/Jan/2022 13:08:43] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_